In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
file_path = '../input/up-crime-data-year-2019/MR data - Compiled Data Set.csv'
df = pd.read_csv(file_path)

#droping title and text columns
x_full = df.drop(axis = 1, labels = ['Title','Text'])


In [ ]:
#Renaming big column names(cleaning)

x_full = x_full.rename(columns = {'Murder Happened with reason:\n1:Property/Land Disputes,\n2:Family Dispute\n3:Petty Quarrels,\n4:Money Disputes, \n5:Personal Vendetta, \n6:Love Affairs, \n7:Casteism\n8: Unknown/other':'Murder Reason', 'Crime Against Women ( Combined ):\n1. Murder with Rape \n2. Dowry Deaths(Sec. 3048)\n3. Suicide(sec 305/306)\n4. Kidnapping(All)\n5. Acid Attack(Sec. 326A IPC)\n6. Cruelty by Husband/in-laws((Sec.498 A IPC)\n7. Rape only(Sec. 376 or 511 IPC)\n8. Assault on Women with Intent to Outrage her Modesty (Sec. 354 IPC)\n9. Cyber Crimes against Women\n10. Protection of Children from Sexual Offences Act':'crime against women(Act)'})
x_full.columns

In [ ]:
#drop rows with all null values apart from 'City '( cleaning )
x_full = x_full.dropna(axis = 0, subset= ['City '], how = 'all')


In [ ]:
#checking Null Values
x_full.isnull().sum()

In [ ]:
#replacing NaN values in "number of" type and NaN values in crime agaisnt women column with 0 
x_full = x_full.fillna(0)

In [ ]:
#seperating acts

crime_acts = x_full['crime against women(Act)'].astype('string').values
for i in range(1,11):
    x_full.insert(8+i,"Act {}".format(i),0)
for j in range(len(crime_acts)):
    splitted = crime_acts[j].split(',')
    for x in splitted:
        if x=='0':
            continue
        x_full.loc[int(j),'Act {}'.format(x.strip())]=1
x_full

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')
sns.set()

# **City-wise Crime Report**

In [ ]:
sns.countplot(x = 'City ', data = x_full, palette = 'mako').set(title = 'city-wise crime count')

# **Reasons For Murder**

In [ ]:

plt.figure(figsize = (5,5))
reason = x_full['Murder Reason'].value_counts().to_dict()
reason.pop(0)
val = list(reason.values())
label = list(reason.keys())
plt.title('Distribution of Reasons for Murder',fontsize = 20)
plt.pie(x = val,labels = label)
plt.show()


### Most of the known murder cases are due to Love Affairs
### Though murder cases due to casteism is least but its still there, stating caste system still prevails in UP


# **Love Affair victims Gender Distribution**

In [ ]:
#love affairs victims
mv = x_full.loc[x_full['Murder Reason'] == 'Love Affairs', "Number of male victims(adult)"].sum()
fv = x_full.loc[x_full['Murder Reason'] == 'Love Affairs', "Number of female victims(adult)"].sum()
plt.figure(figsize = (5,5))
plt.title('Love Affair victims',fontsize = 20)
plt.pie(x = [mv,fv],labels = ['females','males'],colors = ['pink','blue'])
plt.show()

### We see almost equal distribution, which shows both groups are equally victimized

# **What crimes are causing women's death?**

In [ ]:
#what is the biggest cause of women's death
wmr = x_full.loc[x_full['Number of female victims(adult)']>0]
plt.figure(figsize = (15,7))
sns.countplot(x = 'Murder Reason', data = wmr)

### Major cause of women's death is 'Love Affairs' and 'Family Disputes', the former being the highest one

# **Cause of men's death**

In [ ]:
wmr = x_full.loc[x_full['Number of male victims(adult)']>0]
plt.figure(figsize = (15,7))
sns.countplot(x = 'Murder Reason', data = wmr)

### 'Personal Vendetta', 'Love Affairs', 'Money Disputes', all are major causes of men's death

# **Distribution of Crime Againt Women on basis of Acts**

In [ ]:
s = x_full.iloc[:,9:(9+10)].sum()
plt.figure(figsize = (15,5))
plt.bar(s.index,s.values)



> 1. Murder with Rape 
> 2. Dowry Deaths(Sec. 3048)
> 3. Suicide(sec 305/306)
> 4. Kidnapping(All)
> 5. Acid Attack(Sec. 326A IPC)\n6. Cruelty by Husband/in-laws((Sec.498 A IPC)
> 7. Rape only(Sec. 376 or 511 IPC)
> 8. Assault on Women with Intent to Outrage her Modesty (Sec. 354 IPC)
> 9. Cyber Crimes against Women
> 10. Protection of Children from Sexual Offences Act

# **Property Disputes**

In [ ]:

mv = x_full.loc[x_full['Murder Reason'] == 'Property Disputes', "Number of male victims(adult)"].sum()
fv = x_full.loc[x_full['Murder Reason'] == 'Property Disputes', "Number of female victims(adult)"].sum()
plt.figure(figsize = (5,5))
plt.title('Property Disputes victims',fontsize = 20)
plt.pie(x = [mv,fv],labels = ['females','males'],colors = ['pink','blue'])
plt.show()

## More females are murdered in property disputes

# **Female vs Male involvement**

In [ ]:
#females vs males crime
sns.set_palette("BrBG",1)
females = x_full['Number of female(adult)'].sum()
males = x_full['Number of male(adult) '].sum()

sns.barplot(x= ['femlaes','males'], y= [females, males]).set(title = 'Involved in Crime')

# **Children vs Adulit Victims**

In [ ]:
#Children vs Adults victims in crime city-wise
sns.set_palette("RdBu",)

females = x_full['Number of female victims(adult)'].sum()
males = x_full['Number of male victims(adult)'].sum()
child = x_full['Number of child victims'].sum()

sns.barplot(x= ['femlaes(adults)','males(adult)','children'], y= [females, males, child]).set(title = 'Victims of UP Crime')

### We see a higher number of adult male involvement, victimisation in UP crime report

# **City-wise Child victims**

In [ ]:
#child victims city-wise
sns.set_palette("YlOrBr", 1)
gbd = x_full.loc[x_full['City '] == 'Ghaziabad' , 'Number of child victims' ].sum()
lkw = x_full.loc[x_full['City '] == 'Lucknow' , 'Number of child victims' ].sum()
knp = x_full.loc[x_full['City '] == 'Kanpur' , 'Number of child victims' ].sum()

sns.barplot(y = [gbd,lkw,knp], x = ['Ghaziabad','Lucknow', 'Kanpur']).set(title = 'city-wise child victims')


In [ ]:
sns.set_palette("YlOrBr", 1)
gbd = x_full.loc[x_full['City '] == 'Ghaziabad' , 'Kidnnaping:\nNumber of child victims' ].sum()
lkw = x_full.loc[x_full['City '] == 'Lucknow' , 'Kidnnaping:\nNumber of child victims' ].sum()
knp = x_full.loc[x_full['City '] == 'Kanpur' , 'Kidnnaping:\nNumber of child victims' ].sum()

sns.barplot(y = [gbd,lkw,knp], x = ['Ghaziabad','Lucknow', 'Kanpur']).set(title = 'city-wise child kidnappings')

### Ghaziabad has high number of child victims and kidnapings, making the place unsafe for children